In [8]:
# 导入需要的库
import pandas as pd
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from instrument_utility import *
import os 


def ret_df(w,path = 'D:\\Quant_Code\\data\\index\\weekly_sw_3.csv'):
#     path = 'D:\\Quant_Code\\data\\index\\weekly_index.csv'
    
    df = pd.read_csv(path)
    df_list =  list()
    codes = df['code'].unique().tolist()
    for i in codes:
        df_t = df[df['code'] == i]
        df_t.reset_index(drop=True,inplace=True)
        df_t['ret'] =(df_t['close_last']/df_t['close_last'].shift(w)-1)
        df_t= df_t.iloc[w:].fillna(0)
        df_list.append(df_t)
    df_ret = pd.concat(df_list,axis=0,ignore_index=True)
    cols = ['date','code','ret']
    df_ret= df_ret[cols]
    return df_ret
def ret_bar(path,w=1):
    df = ret_df(w,path)
    codes  = df['code'].tolist()
    x_ = list()
    y_ = list()
    for code in codes:
        df_t = df[df['code'] == code]
        y = df_t['ret'].tolist()[-2]
        y_.append(round(y,3))
        x_.append(code)
    # 虚假数据   
   
    bar_ = sorted(dict(zip(x_,y_)).items(),key=lambda x:x[1],reverse=True)
    x_s = list()
    y_s = list()
    iu = InstrumentUtility().swcode2name(level='L2')
    
    for item in bar_:      
        key = item[0]#         
#         print(iu[key])
        x_s.append(iu[key])
        y_s.append(item[1]) 

    chart = Bar(
                init_opts=opts.InitOpts(
            theme='light',
            width='1580px',
            height='1000px'
                )
                    
            )
    chart.add_xaxis(x_s)
    chart.add_yaxis(
        '', 
        y_s,
        # 标签配置
        label_opts=opts.LabelOpts(
            font_size=9,  # 字体大小
            color='black',  # 文字颜色
            font_style='italic',  # 字体风格
            font_weight='bolder',  # 加粗
            font_family='Courier New',  # 字体
            position='insideTop',  # 显示位置
            formatter = '\n{b}\n{c}',  # 文本内容
            rotate= 60
            )
        )
    chart.set_global_opts(
        #标题设置
        title_opts=opts.TitleOpts(
#             title="申万三级收益率水平(按中位数排序)",
            title= str(w) +'周申万二级涨幅',
            
            pos_left='center'
            ),
        # 坐标轴名称设置
        yaxis_opts = opts.AxisOpts(name='收益率'),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True,
            pos_left='75%',
            pos_top='1%',
            orient='horizontal',
            legend_icon='circle'  # 设置图例图形为实心圆
        ),
        datazoom_opts=opts.DataZoomOpts(
            range_start=50,  # 开始范围
            range_end=60,  # 结束范围
            orient='horizontal',  # 设置为垂直布局
            type_='slider',  # slider形式
            is_zoom_lock=True,  # 锁定区域大小
            pos_left='10%'  # 设置位置
            )
    )
    fil_name = r"D:\\Quant_Code\\data\\plot_sw_index\\申万二级收益排名.html"
    chart.render(fil_name)
#     return x_,y_         

    
# df = ret_df(12)
def amount_df(w,path = 'D:\\Quant_Code\\data\\index\\weekly_sw_3.csv'):
#     path = 'D:\\Quant_Code\\data\\index\\weekly_index.csv'
    
    df = pd.read_csv(path)
    df_list =  list()
    codes = df['code'].unique().tolist()
    for i in codes:
        df_t = df[df['code'] == i]
        df_t.reset_index(drop=True,inplace=True)
        df_t['amount'] =(df_t['amount_sum'] - df_t['amount_sum'].shift(w))
        df_t= df_t.iloc[w:].fillna(0)
        df_list.append(df_t)
    df_amount = pd.concat(df_list,axis=0,ignore_index=True)
    cols = ['date','code','amount']
    df_amount= df_amount[cols]
    return df_amount

def index_sw_distribution(df,n):
    # 渐变配色设置代码
    color_js_1 = """
                new echarts.graphic.LinearGradient(
                                    0,
                                    1,
                                    0,
                                    0,
                                    [{offset: 0, color: '#00BFFF'},
                                     {offset: 1, color: '#DA70D6'}],
                                    false)
               """
    chart = Boxplot(
        init_opts=opts.InitOpts(
            theme='light',
            width='1280px',
            height='1000px'
        )
    )
    # 新建一些空的list存放数据
    data_x = []
    data_y = []
    codes = df['code'].unique().tolist()
    for item in codes:
        if 'ret' in df.columns.tolist():
            data_y.append(df[df['code'] == item]['ret'].tolist())
            fil_name = r"D:\\Quant_Code\\data\\plot_sw_index\\申万二级收益分布.html"
            title_str = str(n)+"周申万二级收益率(按中位数排序)"
        else:
            data_y.append(df[df['code'] == item]['amount'].tolist())
            fil_name = r"D:\\Quant_Code\\data\\plot_sw_index\\申万二级资金进出.html"
            title_str = str(n)+"周申万二级资金进出(按中位数排序)"
        data_x.append(item)

    # 箱型图数据处理
    # 计算最大值，最小值，中位数等等
    data_y = chart.prepare_data(data_y)
    # data_y
    data_x_s = []
    data_y_s = []
    #按中位数排序得到最终数据
    data_list = sorted(dict(zip(data_x,data_y)).items(),key=lambda x:x[1][2],reverse=False)
    for item in data_list:        
        data_x_s.append(item[0])
        data_y_s.append(item[1])  
    fin_dic = InstrumentUtility().get_sw_list('L2')
    # print(fin_dic)
    data_x_s_name = list()
    for i in data_x_s:
        data_x_s_name.append(fin_dic[i])
    # print(data_x_s)
    # 添加x轴数据
    chart.add_xaxis(
        data_x_s_name)

    # 添加y轴数据
    chart.add_yaxis(
        '',
        data_y_s,
        itemstyle_opts={  # 图元样式
            'borderColor': JsCode(color_js_1),
            'color': JsCode(color_js_1)
        },
    )

    chart.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
#             title="申万三级收益率水平(按中位数排序)",
            title=title_str,
            
            pos_left='center'
            ),
        # 坐标轴名称设置
        yaxis_opts = opts.AxisOpts(name=''),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True,
            pos_left='75%',
            pos_top='1%',
            orient='horizontal',
            legend_icon='circle'  # 设置图例图形为实心圆
        ),
        datazoom_opts=opts.DataZoomOpts(
            range_start=50,  # 开始范围
            range_end=57,  # 结束范围
            orient='vertical',  # 设置为垂直布局
            type_='slider',  # slider形式
            is_zoom_lock=True,  # 锁定区域大小
            pos_left='1%'  # 设置位置
            )
    )
    # xy轴翻转
    chart.reversal_axis()
    chart.render(fil_name)
    # chart.render_notebook()
if __name__ == '__main__':

    path = 'D:\\Quant_Code\\data\\sw_2_index_weekly\\'+ os.listdir('D:\\Quant_Code\\data\\sw_2_index_weekly')[0]
#     path = 'D:\\Quant_Code\\data\\sw_2_index_weekly\\'+ os.listdir('D:\\Quant_Code\\data\\sw_2_index_weekly')[0]
#     path_3 = 'D:\\Quant_Code\\data\\sw_3_index_weekly\\'+ os.listdir('D:\\Quant_Code\\data\\sw_3_index_weekly')[0]
    
    df = ret_df(12,path)    
    index_sw_distribution(df,12)
    df_t = amount_df(1,path)
    index_sw_distribution(df_t,1)
    ret_bar(path,w=1)
    

    
    
    

C:\Users\rufus\AppData\Local\Temp/ipykernel_14848/509707749.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['ret'] =(df_t['close_last']/df_t['close_last'].shift(w)-1)
C:\Users\rufus\AppData\Local\Temp/ipykernel_14848/509707749.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['amount'] =(df_t['amount_sum'] - df_t['amount_sum'].shift(w))
